In [ ]:
# For package management
workdir_ = '/glade/work/juliob/'
import sys
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cftime
import datetime

# Get some regirdding stuff
from PyRegridding.Drivers import RegridField as RgF
from PyRegridding.Utils import GridUtils as GrU

# Some other useful packages 
import importlib
import copy
#import esmpy as E

# reload dev codes
importlib.reload( RgF )
importlib.reload( GrU )




In [ ]:
def generate_no_leap_dates(start_date, end_date):
    """
    Generate a list of cftime.DatetimeNoLeap dates for a no-leap calendar, from start_date to end_date.
    """
    dates = []
    current_date = start_date
    delta = datetime.timedelta(days=1)  # Use the standard datetime timedelta

    while current_date <= end_date:
        dates.append(current_date)
        # Move to the next day (using datetime timedelta)
        current_date = current_date + delta
    
    return dates



In [ ]:
RgOb={}
# Restriction ... 
src_grid = 'NOAA_OI_SST'
dst_grid = 'ne16pg3'
roo = GrU.regrid_object_lib(RgOb=RgOb, src=src_grid, dst=dst_grid)
# Prolongation ...
dst_grid = 'NOAA_OI_SST'
src_grid = 'ne16pg3'
roo = GrU.regrid_object_lib(RgOb=RgOb, src=src_grid, dst=dst_grid, RegridMethod='BILINEAR' )

##############################
# get SSTs

SST = xr.open_dataset('/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst_ice_NOAA_QxQ_2003_2005.nc' )
lon=SST.lon.values
lat=SST.lat.values
ssts = SST.SST_cpl.values
icec = SST.ice_cov.values

print( " EXtracted data into ndarrays " )

# "Restrict 0.25 SSTs ice_cov to ne16 grid conservatively
Src, Dst ='NOAA_OI_SST', 'ne16pg3' 
rgob = GrU.regrid_object_lib(RgOb=RgOb, src=Src, dst=Dst )
ssts_c2=RgF.Horz(xfld_Src=ssts , Src=Src, Dst=Dst , RegridObj_In=rgob  ) 
icec_c2=RgF.Horz(xfld_Src=icec , Src=Src, Dst=Dst , RegridObj_In=rgob  ) 

print( " 'Restricted' to ne16 " )

# "Prolong" ne16 SSTs ice_cov back to 0.25 grid BILINEARLY
Src, Dst =  'ne16pg3'  , 'NOAA_OI_SST'
rgob = GrU.regrid_object_lib(RgOb=RgOb, src=Src, dst=Dst, RegridMethod='BILINEAR' )
ssts_c2_xQ =RgF.Horz(xfld_Src=ssts_c2 , Src=Src, Dst=Dst , RegridObj_In=rgob  ) 
icec_c2_xQ =RgF.Horz(xfld_Src=icec_c2 , Src=Src, Dst=Dst , RegridObj_In=rgob  ) 

print( " 'Prolonged' back to NOAA QxQ grid " )



# Step 1: Define the start and end date
start_date = cftime.DatetimeNoLeap(2003, 1, 1, 12, 0, 0, has_year_zero=True)
end_date = cftime.DatetimeNoLeap(2005, 12, 31, 12, 0, 0, has_year_zero=True)

# Step 2: Generate daily dates using cftime
cftime_dates = generate_no_leap_dates(start_date, end_date)

# Step 3: Create an xarray DataArray with the generated time dimension
time_da = xr.DataArray(cftime_dates, dims='time', name='time')






ds = xr.Dataset(
    coords={
        'time': time_da,
        'lat': lat ,
        'lon': lon 
    }
)

#######################
print(len( ds.time ) )

ntimes=len( ds.time ) 
date = np.zeros( ntimes, dtype=np.int32 )
datesec = np.zeros( ntimes, dtype=np.int32 )

for n in np.arange( ntimes ):
    date[n] = 10_000*ds.time[n].item().year+100*ds.time[n].item().month+ds.time[n].item().day
    datesec[n] = 3_600*ds.time[n].item().hour

ds['date'] = xr.DataArray( date.astype(np.int32) ,
                             dims=('time',)) 
ds['datesec'] = xr.DataArray( datesec.astype(np.int32) ,
                             dims=('time',)) 








ds['SST_cpl'] = xr.DataArray( ssts_c2_xQ.astype(np.float32)  , 
                             dims=('time','lat','lon',)) 
ds['ice_cov'] = xr.DataArray( icec_c2_xQ.astype(np.float32) ,
                             dims=('time','lat','lon',)) 

fileN = '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/ssticeNOAA_sm_2003_2005.nc' 
ds.to_netcdf( fileN )


print( f" wrote smoothed SST/Ice data set to {fileN}")

In [ ]:
"""
SST_sm = copy.deepcopy( SST )
SST_sm['SST_cpl'].values = ssts_c2_xQ
SST_sm['ice_cov'].values = icec_c2_xQ
SST_sm.to_netcdf( '/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst_ice_NOAA_QxQ_smooth_2003_2005.nc' )
"""

In [ ]:
noaa=GrU.gridInfo( 'NOAA_OI_SST' )
scrip_file=noaa['scrip']
Scr_noaa=xr.open_dataset( scrip_file )
Scr_noaa
area=Scr_noaa.grid_area.values.reshape( 720, 1440)
plt.contourf( area )

In [ ]:
hpssts=ssts - ssts_c2_xQ

In [ ]:

clev = np.linspace(-10,40,num=51)
dlev = np.linspace(-1,1,num=11)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(27, 5))
#hpssts=ssts - ssts_c2_xQ

# Flatten axs to easily access each subplot by index
ax = ax.flatten()


t0,t1=0,90
co=ax[0].contourf( np.average( ssts[t0:t1,:,:],axis=0 ) , levels = clev , cmap='plasma' )
cb=plt.colorbar( co )

co=ax[1].contourf( np.average( ssts_c2_xQ[t0:t1,:,:],axis=0 ) , levels = clev , cmap='plasma' )
cb=plt.colorbar( co )

co=ax[2].contourf( np.average( hpssts[t0:t1,:,:],axis=0 ) , levels = dlev , cmap='bwr' )
cb=plt.colorbar( co )





In [ ]:
plt.plot( np.average(ssts[500,:,:],axis=1) )
plt.plot( np.average(ssts_c2_xQ[500,:,:],axis=1) )


In [ ]:
hpssts2 = hpssts**2

In [ ]:

print( np.sum( hpssts[500,:,:]*area )/ np.sum( area ) )
print( np.sqrt( np.sum( hpssts2[500,:,:]*area )/ np.sum( area ) ) )
